## py_collector

Generic file, api, and data monitoring system. Platform and db agnostic.



In [ ]:
from datetime import datetime, timezone, timedelta
from threading import Thread, Event
import threading

class Scheduler(Thread):
    ''' Controls scheduling for py_curator
    
    Based havily on Timer implementation here
    https://github.com/python/cpython/blob/3.9/Lib/threading.py
    '''
    initial_run = True
    
    def __init__(self, start_time=None, milliseconds=0, seconds=0, days=0,minutes=0, weeks=0,
                 timezone = None, count=1, separator=1 ):
        
        Thread.__init__(self)
        self.finished = Event()
        self.count = count
        self.separator_delta =separator
        start_time = datetime.now() if start_time == None else start_time
        
        self.start_diff = abs(round((datetime.now() - start_time).total_seconds(),3))
        self.delta = timedelta(weeks=weeks, 
                               days=days, 
                               minutes=minutes, 
                               seconds=seconds, 
                               milliseconds=milliseconds).total_seconds()
    
    def cancel(self):
        self.finished.set()
        
    def init_run(self, func):
        ''' initial run to handle the wait'''
        if self.initial_run:
            self.initial_run = False
            self.finished.wait(self.start_diff)
            func()
            
    def schedule(self, func):
        ''' takes in a last run datetime, if the specified time interval + last run = 
        _now, return true, otherwise false'''
        
        self.init_run(func)
            
        self.finished.wait(self.delta)
        for i in range(self.count):
            func()
            self.finished.wait(self.separator_delta)
            
        self.schedule(func)

class Collector:
    
    def upload(self)->None:
        ''' What errors should be here?'''
        raise NotImplementedError('Please see documentation for implementation example')
    
    def is_new(self)->bool:
        raise NotImplementedError('Please see documentation for implementation example')
        
    def orchestrate(self):
        if self.is_new():
            self.upload()
            self.last_run = datetime.now()
        
    def monitor(self)->None:
        
        self.scheduler.schedule(self.orchestrate)
        

In [20]:
class test(Collector):
    #start_time = datetime(year=2021, month=6, day=28, hour=21, minute=39, second=0)
    scheduler = Scheduler(seconds=3, count=1, separator=1)
    
    def upload(self):
        print(datetime.now())
        print('OG Collector')
        
    def is_new(self):
        return True
    
class test1(Collector):
    scheduler = Scheduler(seconds=3, count=1, separator=1)
    
    def upload(self):
        print('Collector 1')
        
    def is_new(self):
        return True

In [21]:
t0 = test()
t1 = test1()

2021-06-28 21:06:44.041850
Loaded!
2021-06-28 21:06:44.047062
Loaded test1!!


In [24]:
th1 = threading.Thread( target=t0.monitor, daemon = True)
th2 = threading.Thread( target=t1.monitor, daemon = True)

SyntaxError: invalid syntax (<ipython-input-24-2d6bb8008619>, line 1)

2021-06-28 21:08:32.186691
Loaded test1!!
2021-06-28 21:08:32.190556
Loaded!
2021-06-28 21:08:36.189354
Loaded test1!!
2021-06-28 21:08:36.193769
Loaded!
2021-06-28 21:08:40.194078
Loaded test1!!
2021-06-28 21:08:40.195780
Loaded!
2021-06-28 21:08:44.203017
Loaded test1!!
2021-06-28 21:08:44.203529
Loaded!
2021-06-28 21:08:48.206958
Loaded test1!!
2021-06-28 21:08:48.207646
Loaded!
2021-06-28 21:08:52.213107
Loaded test1!!
2021-06-28 21:08:52.217072
Loaded!
2021-06-28 21:08:56.220448
Loaded test1!!
2021-06-28 21:08:56.222903
Loaded!
2021-06-28 21:09:00.2289872021-06-28 21:09:00.229115
Loaded test1!!

Loaded!
2021-06-28 21:09:04.235997
Loaded test1!!
2021-06-28 21:09:04.238548
Loaded!
2021-06-28 21:09:08.242383
Loaded test1!!
2021-06-28 21:09:08.244044
Loaded!
2021-06-28 21:09:12.2517462021-06-28 21:09:12.251911
Loaded!

Loaded test1!!
2021-06-28 21:09:16.2551242021-06-28 21:09:16.255238
Loaded!

Loaded test1!!
2021-06-28 21:09:20.2629752021-06-28 21:09:20.263229
Loaded test1!!

Loaded!

In [18]:
th1.start()
th2.start()

2021-06-28 21:05:52.956923
Loaded test1!!
2021-06-28 21:05:55.960961
Loaded!
2021-06-28 21:05:55.961903
Loaded test1!!
2021-06-28 21:05:59.9648752021-06-28 21:05:59.965175
Loaded!

Loaded test1!!


In [ ]:
th1.start()
th2.start()for i in range(2):
    print(i)

In [ ]:
th1.join()
th2.start()

In [23]:
dir(th1)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_args',
 '_bootstrap',
 '_bootstrap_inner',
 '_daemonic',
 '_delete',
 '_exc_info',
 '_ident',
 '_initialized',
 '_is_stopped',
 '_kwargs',
 '_name',
 '_reset_internal_locks',
 '_set_ident',
 '_set_tstate_lock',
 '_started',
 '_stderr',
 '_stop',
 '_target',
 '_tstate_lock',
 '_wait_for_tstate_lock',
 'daemon',
 'getName',
 'ident',
 'isAlive',
 'isDaemon',
 'is_alive',
 'join',
 'name',
 'run',
 'setDaemon',
 'setName',
 'start']

2021-06-28 21:07:20.097173
Loaded!
2021-06-28 21:07:20.106270
Loaded test1!!
2021-06-28 21:07:24.104522
Loaded!
2021-06-28 21:07:24.112381
Loaded test1!!
2021-06-28 21:07:28.114987
Loaded!
2021-06-28 21:07:28.119772
Loaded test1!!
2021-06-28 21:07:32.121380
Loaded!
2021-06-28 21:07:32.122984
Loaded test1!!
2021-06-28 21:07:36.128952
Loaded!
2021-06-28 21:07:36.130851
Loaded test1!!
2021-06-28 21:07:40.133780
Loaded!
2021-06-28 21:07:40.136271
Loaded test1!!
2021-06-28 21:07:44.137595
Loaded!
2021-06-28 21:07:44.140003
Loaded test1!!
2021-06-28 21:07:48.138961
Loaded!
2021-06-28 21:07:48.143655
Loaded test1!!
2021-06-28 21:07:52.143123
Loaded!
2021-06-28 21:07:52.147386
Loaded test1!!
2021-06-28 21:07:56.146831
Loaded!
2021-06-28 21:07:56.151836
Loaded test1!!
2021-06-28 21:08:00.149205
Loaded!
2021-06-28 21:08:00.160858
Loaded test1!!
2021-06-28 21:08:04.154187
Loaded!
2021-06-28 21:08:04.161540
Loaded test1!!
2021-06-28 21:08:08.161094
Loaded!
2021-06-28 21:08:08.166151
Loaded test1!!